# <div align="center" style="color: #ff5733;">TSA Closure Third Notification</div>

In [1]:
# Import Libraries   

import pandas as pd  
import numpy as np  

from google.cloud import bigquery
import os

path = r"C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json"

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path
client = bigquery.Client(project="prj-prod-dataplatform")

# Ask Balasubramani to refresh risk_credit_mis.tsa_account_closure table.
**Check if that is refreshed when Balasubramani confirms it**

In [2]:
sq = """select Run_Date, count(1)cnt from risk_credit_mis.tsa_account_closure group by 1 order by 1 desc"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
df.head(1)

Job ID 02cace4d-6ef7-4047-b526-b0a19c1b263e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


,Run_Date,cnt
0,2025-02-06,753083


The date should show yesterday's date

# Read the data from the first sheet for Type1 and Type2 data provided by Marvin

In [3]:
d1 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\December2024_January2025\ThirdNotification_files\Type 1  & 2  - Dec 2024.xlsx")
d1.shape

(35554, 2)

In [4]:
print(f"The unique count of customer_id is:\t{len(d1['Customer_Id'].unique())}")
d1.head()


The unique count of customer_id is:	35554


,Customer_Id,ACTION_TYPE
0,2982912,TYPE 1
1,2911320,TYPE 1
2,2877112,TYPE 1
3,2984139,TYPE 1
4,2910719,TYPE 1


In [5]:
d1.groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    35507
TYPE 2       47
Name: Customer_Id, dtype: int64

# Read the data from the second sheet for Type3 data provided by Marvin

In [6]:
d2 = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\December2024_January2025\ThirdNotification_files\Type 3  - Dec 2024.xlsx")
d2.shape

(1458, 2)

In [7]:
print(f"The unique count of customer_id is:\t{len(d2['Customer_Id'].unique())}")
d2.head()

The unique count of customer_id is:	1458


,Customer_Id,ACTION_TYPE
0,1360717,TYPE 3
1,1404801,TYPE 3
2,1083868,TYPE 3
3,1310029,TYPE 3
4,1805856,TYPE 3


In [8]:
d2.groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 3    1458
Name: Customer_Id, dtype: int64

In [9]:
result = pd.concat([d1, d2], axis=0, ignore_index=True)
result.shape

(37012, 2)

In [10]:
result.ACTION_TYPE.value_counts()

ACTION_TYPE
TYPE 1    35507
TYPE 3     1458
TYPE 2       47
Name: count, dtype: int64

In [14]:
result[result['Customer_Id'] == '1083868']

,Customer_Id,ACTION_TYPE
35556,1083868,TYPE 3


In [12]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37012 entries, 0 to 37011
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  37012 non-null  int64 
 1   ACTION_TYPE  37012 non-null  object
dtypes: int64(1), object(1)
memory usage: 578.4+ KB


In [13]:
result['Customer_Id'] = result['Customer_Id'].astype(str)
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37012 entries, 0 to 37011
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Customer_Id  37012 non-null  object
 1   ACTION_TYPE  37012 non-null  object
dtypes: object(2)
memory usage: 578.4+ KB


In [15]:
result.to_csv("temp.csv", index = False)

In [17]:
# Define the dataset and table references

dataset_id = 'dap_ds_poweruser_playground'
table_id = 'temp_tsa_secondnotificationcustomers'
# Define the table schema as per your DataFrame columns
schema = [
    bigquery.SchemaField("Customer_Id", "STRING"),
    bigquery.SchemaField("ACTION_TYPE", "STRING"),
]
# Create the dataset reference
dataset_ref = client.dataset(dataset_id)
# Define the table reference
table_ref = dataset_ref.table(table_id)
# Configure the job to overwrite the table if it already exists
job_config = bigquery.LoadJobConfig()
# Load the DataFrame into BigQuery
job = client.load_table_from_dataframe(result, table_ref, job_config=job_config)
# Wait for the job to complete
job.result()
print(f"Table {table_id} created in dataset {dataset_id}.")

Table temp_tsa_secondnotificationcustomers created in dataset dap_ds_poweruser_playground.


In [18]:
sq = """select * from  dap_ds_poweruser_playground.temp_tsa_secondnotificationcustomers;"""

a = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 1cd6ce93-0846-4f1c-a7f5-1adbb91aaf7e successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [21]:
print("The total breakdown is:")
print(a.groupby('ACTION_TYPE')['Customer_Id'].count())
print("The unique breakdown is:")
print(a.groupby('ACTION_TYPE')['Customer_Id'].nunique())

The total breakdown is:
ACTION_TYPE
TYPE 1    35507
TYPE 2       47
TYPE 3     1458
Name: Customer_Id, dtype: int64
The unique breakdown is:
ACTION_TYPE
TYPE 1    35507
TYPE 2       47
TYPE 3     1458
Name: Customer_Id, dtype: int64


In [16]:
# a = tuple(result['Customer_Id'])
# print(a[:5])
# print(len(a))

('2982912', '2911320', '2877112', '2984139', '2910719')
37012


In [22]:
d = '2025-02-06'   ### Need to change this to D-1 everytime we run this code ###
# Get the current date
current_date = pd.Timestamp.now().date()
d_minus_1_days = current_date - pd.DateOffset(days=1)
# Calculate D-60 days
d_minus_60_days = '2024-11-08'
# Calculate D-180 days
d_minus_180_days = '2024-07-11'

print("D-1 day from current date", d_minus_1_days )
print("D-60 day from the current date - 1:", d_minus_60_days)
print("D-180 day from the current date -1:", d_minus_180_days)

d_minus_1_days_string = d_minus_1_days.strftime('%Y-%m-%d')
print("D-1 day from the current date:", d_minus_1_days_string)

d_minus_60_days_string = d_minus_60_days
print("D-60 day from the current date:", d_minus_60_days_string)

d_minus_180_days_string = d_minus_180_days
print("D-180 day from the current date:", d_minus_180_days_string)

D-1 day from current date 2025-02-06 00:00:00
D-60 day from the current date - 1: 2024-11-08
D-180 day from the current date -1: 2024-07-11
D-1 day from the current date: 2025-02-06
D-60 day from the current date: 2024-11-08
D-180 day from the current date: 2024-07-11


Step1: At line 11 change the date to D-1<br><br>
Step2: At line around 36 (and (TSA_Opening_date < date '2023-10-18')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step3: At line around 45 (WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '2023-10-18')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) Change the date D-60 Days <br><br>
Step4: At line 58 (and TSA_Opening_date < date '2023-06-18'  and (Last_TSA_Credit_Date < date '2023-06-18' and Last_TSA_Debit_Date < date '2023-06-18' and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '2023-06-18')  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)) <br><br>
Step5: At line 72 (Run_Date = date '2024-01-09'###DATE change here to D-1) Change the date to D-1<br><br>

First Notification we change all the steps <br><br>
Second and Third Notification we only change Step1 and Step5
For Second Notificaiton and Third Notificaiton the customre id should be from the file sent by Marvin. <br><br>

In [24]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}' ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  customer_id in (select distinct cast(Customer_Id as string) from  dap_ds_poweruser_playground.temp_tsa_secondnotificationcustomers)
  )
select * from w_tsa;
"""

dfd = client.query(sq).to_dataframe(progress_bar_type =  'tqdm')
dfd.shape

Job ID 5971f501-92db-4d00-957d-88a99267f054 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(37011, 29)

In [29]:
sq = f"""
with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  customer_id in (select distinct cast(Customer_Id as string) from  dap_ds_poweruser_playground.temp_tsa_secondnotificationcustomers)
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Deposit_Balance,a.Active_Term_Deposit_flag,a.Last_TSA_Credit_Date,a.Last_TSA_Debit_Date,
  a.ARI_Flag,a.Active_Loan,a.Total_Loan_Outstanding,  case when b.customerId is not null then 1 else 0 end as Approved_Loans,
  a.Pending_Loan_Applications,a.Hard_Reject_Loans_App,a.Soft_Reject_Loans_App,a.Cancelled_Loans_App, extract(date from a.Last_Login_Date) as Last_Login_Date,
  a.Last_TSA_EOD_Balance_Date,a.Last_Group_Stash_Credit_Date,a.Group_Stash_Balance_Owner,a.Run_Date,a.Batch,a.Processing_Month,a.pending_loans_not_expired_not_exempt,a.expired_loans,
  a.pending_loans_lastDt,a.expired_loans_lastDt,
  CASE 
    WHEN tsa_status = 'Active' and (Last_TSA_Credit_Date = date '1970-01-01' and Last_Group_Stash_Credit_Date is null)
    and (TSA_Opening_date < date '{d_minus_60_days_string}')   ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan,0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)  THEN 'TYPE 1'
    and b.customerId is null  THEN 'TYPE 1'


    WHEN tsa_status = 'Active' and Last_TSA_EOD_Balance_Date is null and (TSA_Opening_date < date '{d_minus_60_days_string}')  ####D-60 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
    and ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) = 0 )
    and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
    and (Active_Term_Deposit_flag = 0)
    and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
    and b.customerId is null  
    and (date_trunc(TSA_Opening_date,day) = date_trunc(Last_Login_Date,day)) THEN 'TYPE 2'

    
    WHEN ( round(coalesce(TSA_Balance,0) + coalesce(Solo_Stash_Balance,0) + coalesce(Group_Stash_Balance,0) + coalesce(Term_Deposit_Balance,0),2) < 1 )
            and (coalesce(Active_Loan, 0) = 0) and (COALESCE(Pending_Loan_Applications,0) = 0)
            and (Active_Term_Deposit_flag = 0)
            and TSA_Opening_date < date '{d_minus_180_days_string}'                                     ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and (Last_TSA_Credit_Date < date '{d_minus_180_days_string}'                                 ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and Last_TSA_Debit_Date < date '{d_minus_180_days_string}'                                         ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and coalesce(Last_Group_Stash_Credit_Date,date '1970-01-01') < date '{d_minus_180_days_string}' )  ####D-180 days(SHOULD BE SAME DATE FOR 1ST,2ND,3RD NOTIFICATION)
            and tsa_status = 'Active'
            and (ARI_Flag = 'N')
    --and (Approved_Loans = 0)
            and b.customerId is null  
    THEN 'TYPE 3'
    ELSE 'NO ACTION'
    END AS ACTION_TYPE
  from w_tsa A
  left join w_approved b
  on cast(a.Customer_Id as int64) = b.customerId
  left join core_raw.customer_accounts c
  on c.OFCUSTOMERID = a.Customer_Id and c.CRINTERDESC = 'Transactional Savings Account Inv_R' and c.OFISCLOSED = 'N'
  where
  Run_Date = date '{d_minus_1_days_string}'  ###DATE change here to D-1
)
"""


In [30]:
print(sq)


with
w_kyc
as
(
  select distinct PARTYID, SEGCATEGORYDESC, row_number() over(partition by partyid order by SEGEFFECTIVEFROM desc) as rnk from `core_raw.party_details`
),
w_tsa
as
(
  select * from risk_credit_mis.tsa_account_closure where Run_Date = date '2025-02-06'  ###DATE change here to D-1
  and
  Customer_Id not in (select cast(PARTYID as string) from w_kyc where rnk = 1 and SEGCATEGORYDESC = 'BKYC')
  and
  customer_id in (select distinct cast(Customer_Id as string) from  dap_ds_poweruser_playground.temp_tsa_secondnotificationcustomers)
),
w_approved
as
(
  select DISTINCT customerId from risk_credit_mis.loan_master_table where loantype = 'BIGLOAN' OR FLAGAPPROVAL = 1 OR FLAGDISBURSEMENT = 1  
  or (stage1Status = 'EXPIRED' AND cddInitiatedDateAndTime IS NULL)
)
select *
from
(
  select distinct
  a.Customer_Id,a.TSA_Opening_date,
  c.OFSTANDARDACCOUNTID as TSA_Account_Number, c.OfblockAmount, 
  a.TSA_Status,a.TSA_Balance,a.Solo_Stash_Balance,a.Group_Stash_Balance,a.Term_Depo

In [31]:
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')





Job ID dba37e1c-0ddf-4648-bbc6-19befad62fca successfully executed: 100%|██████████|



Query is running:   0%|          |


Downloading: 100%|██████████|


In [32]:
pd.set_option('display.max_columns', None)
df.sample(10)

,Customer_Id,TSA_Opening_date,TSA_Account_Number,OfblockAmount,TSA_Status,TSA_Balance,Solo_Stash_Balance,Group_Stash_Balance,Term_Deposit_Balance,Active_Term_Deposit_flag,Last_TSA_Credit_Date,Last_TSA_Debit_Date,ARI_Flag,Active_Loan,Total_Loan_Outstanding,Approved_Loans,Pending_Loan_Applications,Hard_Reject_Loans_App,Soft_Reject_Loans_App,Cancelled_Loans_App,Last_Login_Date,Last_TSA_EOD_Balance_Date,Last_Group_Stash_Credit_Date,Group_Stash_Balance_Owner,Run_Date,Batch,Processing_Month,pending_loans_not_expired_not_exempt,expired_loans,pending_loans_lastDt,expired_loans_lastDt,ACTION_TYPE
17961,2872684,2024-09-26,60828726840007,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-10,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,1,NaT,2024-12-26,TYPE 1
23458,2948373,2024-10-17,60829483730001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-17,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,1,NaT,2025-01-16,TYPE 1
4968,2968218,2024-10-24,60829682180001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-26,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,0,NaT,NaT,TYPE 1
32636,1288116,2022-03-26,60812881160006,0.0,Active,0.89,0.0,0.0,0.0,0,2025-01-31,2024-02-01,N,0,0.0,0,0,0,0,0,2024-02-05,2025-02-06,NaT,NaN,2025-02-06,Batch_1,February,0,3,NaT,2024-05-06,NO ACTION
4182,2913185,2024-10-05,60829131850003,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-05,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,0,NaT,NaT,TYPE 1
28802,2824547,2024-11-06,60828245470002,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-11-06,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,1,NaT,2024-12-05,TYPE 1
32498,2975378,2024-10-26,60829753780008,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-11-28,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,1,NaT,2025-01-25,TYPE 1
14274,2978522,2024-10-27,60829785220001,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-11-29,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,1,NaT,2025-01-26,TYPE 1
2736,2946304,2024-10-16,60829463040009,0.0,Active,0.00,0.0,0.0,0.0,0,1970-01-01,1970-01-01,N,0,0.0,0,0,0,0,0,2024-10-16,NaT,NaT,NaN,2025-02-06,Batch_1,February,0,0,NaT,NaT,TYPE 1
8699,2098985,2023-06-18,60820989850003,0.0,Active,0.24,0.0,0.0,0.0,0,2024-06-30,2024-06-30,N,0,0.0,0,0,0,0,0,2024-10-11,2025-02-06,NaT,NaN,2025-02-06,Batch_1,February,0,0,NaT,NaT,TYPE 3


In [33]:
df.shape

(37011, 32)

In [34]:
df['ACTION_TYPE'].value_counts()

ACTION_TYPE
TYPE 1       35337
TYPE 3        1312
NO ACTION      316
TYPE 2          46
Name: count, dtype: int64

In [35]:
df.to_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\December2024_January2025\ThirdNotification_files\20250207_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification', index = False)

In [36]:
df = pd.read_excel(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\Marvin\TSA Account Closure\December2024_January2025\ThirdNotification_files\20250207_TSA_Account_Closure_3rdtNotification.xlsx", sheet_name='TSA_Closure_3rdNotification')
df.shape

(37011, 32)

In [37]:
df.columns

Index(['Customer_Id', 'TSA_Opening_date', 'TSA_Account_Number',
       'OfblockAmount', 'TSA_Status', 'TSA_Balance', 'Solo_Stash_Balance',
       'Group_Stash_Balance', 'Term_Deposit_Balance',
       'Active_Term_Deposit_flag', 'Last_TSA_Credit_Date',
       'Last_TSA_Debit_Date', 'ARI_Flag', 'Active_Loan',
       'Total_Loan_Outstanding', 'Approved_Loans', 'Pending_Loan_Applications',
       'Hard_Reject_Loans_App', 'Soft_Reject_Loans_App', 'Cancelled_Loans_App',
       'Last_Login_Date', 'Last_TSA_EOD_Balance_Date',
       'Last_Group_Stash_Credit_Date', 'Group_Stash_Balance_Owner', 'Run_Date',
       'Batch', 'Processing_Month', 'pending_loans_not_expired_not_exempt',
       'expired_loans', 'pending_loans_lastDt', 'expired_loans_lastDt',
       'ACTION_TYPE'],
      dtype='object')

In [42]:
df[df['ACTION_TYPE']!='NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique()

ACTION_TYPE
TYPE 1    35337
TYPE 2       46
TYPE 3     1312
Name: Customer_Id, dtype: int64

In [43]:
total_unique_customers = df[df['ACTION_TYPE'] != 'NO ACTION'].groupby('ACTION_TYPE')['Customer_Id'].nunique().sum()
print(total_unique_customers)

36695


In [44]:
df.groupby('ACTION_TYPE')['Customer_Id'].count()

ACTION_TYPE
NO ACTION      316
TYPE 1       35337
TYPE 2          46
TYPE 3        1312
Name: Customer_Id, dtype: int64

In [45]:
total_count_customers = df.groupby('ACTION_TYPE')['Customer_Id'].count().sum()

In [46]:
print(total_count_customers)

37011


In [47]:
len(df['Customer_Id'].unique())

37011